In [ ]:
import sys
from joblib import load

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

import torch
from torch import optim

from reimplemented_approaches.proactive_conformance_checking.models import LSTMCollectiveIDP
from reimplemented_approaches.proactive_conformance_checking.training import Training
from reimplemented_approaches.proactive_conformance_checking.data_prep_split_encode import PrefixDataset


In [ ]:
# Load encoders:
# Load prepared and encoded datasets
train_set, val_set, test_set = PrefixDataset.load_datasets(save_path="../../../data_preparation/Repair/collective/")

print(train_set.tensors[0].size())
print(train_set.tensors[1].size())
print(train_set.tensors[2].size())
print(train_set.tensors[3].size())
print(train_set.tensors[4].size())

encoders = load("../../../data_preparation/Repair/collective/encoders.pkl")
print(encoders)

activity_ids = encoders.get('activity_ids') 
activity_ids_vocab_size_with_default = len(list(activity_ids.keys())) + 1 
print("Activities: ", activity_ids_vocab_size_with_default)

resource_ids = encoders.get('resource_ids')
resource_ids_vocab_size_with_default = len(list(resource_ids.keys())) + 1
print("Resources: ", resource_ids_vocab_size_with_default)

month_ids = encoders.get('month_ids') 
month_ids_vocab_size_with_default = len(list(month_ids.keys())) + 1

print("Months: ", month_ids_vocab_size_with_default)

number_trace_attr = train_set.tensors[3].size(1)
print("Number trace attributes: ", number_trace_attr)

number_deviations_y = len([d for d in encoders.get('deviations')])
print("Number deviatons y: ", number_deviations_y)


In [ ]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print("Using device:", device)

In [ ]:
# Embedding
embedding_dim = 16
# lstm hidden size
lstm_hidden = 128
# fully connected hidden
fc_hidden = 128
# dropout probability
p_dropout = 0.1

model = LSTMCollectiveIDP(activity_vocab_size=activity_ids_vocab_size_with_default,
                          resource_vocab_size=resource_ids_vocab_size_with_default,
                          month_vocab_size=month_ids_vocab_size_with_default,
                          num_trace_features=number_trace_attr,
                          num_output_labels=number_deviations_y,
                          # hyperparams provided in paper:
                          embedding_dim=embedding_dim,
                          lstm_hidden=lstm_hidden,
                          fc_hidden=fc_hidden,
                          dropout=p_dropout,
                          device=device)

In [ ]:
# not mentioned in the paper:
# from code in paper
batch_size=128
# from code in paper
shuffle = True
epochs = 300 # 300 if early stopping (20% val from all train)
# from code in paper
learning_rate = 0.0001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

optimizer_values = {"optimizer":optimizer,
                    "epochs":epochs,
                    "mini_batches":batch_size,
                    "shuffle": shuffle}

# Training with ealy stopping according to journal paper: patience:10, min_delta = 0
training = Training(model=model,
                    train_set=train_set,
                    val_set=val_set,
                    optimizer_values=optimizer_values,
                    loss_mode = 'collective',
                    device=device,
                    saving_path='./LSTM_collecctive_IDP.pkl')

history = training.train()